# XNLI
1. train set 下载地址: 
2. dev and test set 下载地址: https://dl.fbaipublicfiles.com/XNLI/XNLI-1.0.zip (`xnli.dev.tsv`, `xnli.test.tsv`)
3. 这里我已经将数据集下载到本地（`ChineseBERT-Paddle/data/XNLI`）

In [1]:
import sys
sys.path.append('/data1/workspace/research/ChineseBERT-Paddle/Paddle_ChineseBert/PaddleNLP')

In [2]:
from functools import partial
import argparse
import os
import random
import time

import numpy as np
import paddle
import paddle.nn.functional as F
import paddlenlp as ppnlp
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import LinearDecayWithWarmup

In [3]:
from paddlenlp.datasets import load_dataset

def read_train_ds(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        # 跳过列名
        head = None
        for line in f:
            data = line.strip().split('\t', 2)
            if not head:
                head = data
            else:
                sentence1, sentence2, label = data
                yield {"sentence1": sentence1, "sentence2": sentence2, "label": label}

def read(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        # 跳过列名
        head = None
        for line in f:
            data = line.strip().split('\t')
            if not head:
                head = data
            else:
                lan = data[0]
                label = data[1]
                sentence1 = data[6]
                sentence2 = data[7]     
                if lan != 'zh':
                    continue
                else:
                    yield {"sentence1": sentence1, "sentence2": sentence2, "label": label}



In [4]:
train_ds = load_dataset(read_train_ds, data_path='./data/XNLI/xnli.train.tsv', lazy=False)
dev_ds = load_dataset(read, data_path='./data/XNLI/xnli.dev.tsv', lazy=False)
test_ds = load_dataset(read, data_path='./data/XNLI/xnli.test.tsv', lazy=False)

In [5]:
train_ds[:3], dev_ds[:3], test_ds[:3]

([{'sentence1': '从 概念 上 看 , 奶油 收入 有 两 个 基本 方面 产品 和 地理 .',
   'sentence2': '产品 和 地理 是 什么 使 奶油 抹 霜 工作 .',
   'label': 'neutral'},
  {'sentence1': '你 知道 在 这个 季节 , 我 猜 在 你 的 水平 你 把 他们 丢到 下 一个 水平 , 如果 他们 决定 召回 的 家长 队 , 勇士 队 决定 打电话 召回 一个 家伙 从 三 个 a , 然后 一个 双人 上 去. 取代 他 和 一个 男人 去 取代 他',
   'sentence2': '如果 人们 记得 的 话 , 你 就 会 把 事情 弄 丢 了 .',
   'label': 'entailment'},
  {'sentence1': '我们 的 一个 号码 会 非常 详细 地 执行 你 的 指示',
   'sentence2': '我 团队 的 一个 成员 将 非常 精确 地 执行 你 的 命令',
   'label': 'entailment'}],
 [{'sentence1': '他说，妈妈，我回来了。',
   'sentence2': '校车把他放下后，他立即给他妈妈打了电话。',
   'label': 'neutral'},
  {'sentence1': '他说，妈妈，我回来了。',
   'sentence2': '他没说一句话。',
   'label': 'contradiction'},
  {'sentence1': '他说，妈妈，我回来了。',
   'sentence2': '他告诉他的妈妈他已经回到家了。',
   'label': 'entailment'}],
 [{'sentence1': '嗯，我根本没想过，但是我很沮丧，最后我又和他说话了。',
   'sentence2': '我还没有和他再次谈论。',
   'label': 'contradiction'},
  {'sentence1': '嗯，我根本没想过，但是我很沮丧，最后我又和他说话了。',
   'sentence2': '我非常沮丧，我刚刚开始跟他说话。',
   'label': 'entailment'},
  {'sentence1': 

In [6]:
def convert_example(example, tokenizer, max_seq_length=512, is_test=False):
    
    # 【FOCUS】 --> https://github.com/ShannonAI/ChineseBert/blob/main/datasets/xnli_dataset.py
    label_map = {"entailment": 0, "neutral": 1, "contradiction": 2, "contradictory": 2}
    first, second, third = example['sentence1'], example['sentence2'], example['label']

    first_input_tokens = tokenizer.tokenize(first)
    first_pinyin_tokens = tokenizer.convert_sentence_to_pinyin_ids(first, with_specail_token=False)
    
    second_input_tokens = tokenizer.tokenize(second)
    second_pinyin_tokens = tokenizer.convert_sentence_to_pinyin_ids(second, with_specail_token=False)

    label = np.array([label_map[third]], dtype="int64")
    
    # convert sentence to id
    bert_tokens = tokenizer.convert_tokens_to_ids(first_input_tokens) + [102] + tokenizer.convert_tokens_to_ids(second_input_tokens)
    pinyin_tokens = first_pinyin_tokens + [[0] * 8] + second_pinyin_tokens
    if len(bert_tokens) > max_seq_length - 2:
        bert_tokens = bert_tokens[:max_seq_length - 2]
        pinyin_tokens = pinyin_tokens[:max_seq_length - 2]
    # assert
    assert len(bert_tokens) <= max_seq_length
    assert len(bert_tokens) == len(pinyin_tokens)

    input_ids = [101] + bert_tokens + [102]
    pinyin_ids = [[0] * 8] + pinyin_tokens + [[0] * 8]
    
    input_ids = np.array(input_ids)
    pinyin_ids = np.array(pinyin_ids)
    

    return input_ids, pinyin_ids, label

def create_dataloader(dataset,
                      mode='train',
                      batch_size=1,
                      batchify_fn=None,
                      trans_fn=None):
    if trans_fn:
        dataset = dataset.map(trans_fn)

    shuffle = True if mode == 'train' else False
    # shuffle = False
    if mode == 'train':
        batch_sampler = paddle.io.DistributedBatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        batch_sampler = paddle.io.BatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)

    return paddle.io.DataLoader(
        dataset=dataset,
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)


def set_seed(seed):
    """sets random seed"""
    random.seed(seed)
    np.random.seed(seed)
    paddle.seed(seed)


@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader):
    """
    Given a dataset, it evals model and computes the metric.
    Args:
        model(obj:`paddle.nn.Layer`): A model to classify texts.
        data_loader(obj:`paddle.io.DataLoader`): The dataset loader which generates batches.
        criterion(obj:`paddle.nn.Layer`): It can compute the loss.
        metric(obj:`paddle.metric.Metric`): The evaluation metric.
    """
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, pinyin_ids, labels = batch
        logits = model(input_ids, pinyin_ids)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()


def do_train(model, tokenizer):
    paddle.set_device(args.device)
    rank = paddle.distributed.get_rank()
    if paddle.distributed.get_world_size() > 1:
        paddle.distributed.init_parallel_env()

    set_seed(args.seed)

    trans_func = partial(
        convert_example,
        tokenizer=tokenizer)
    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
        Stack(dtype="int64")  # label
    ): [data for data in fn(samples)]
    train_data_loader = create_dataloader(
        train_ds,
        mode='train',
        batch_size=args.batch_size,
        batchify_fn=batchify_fn,
        trans_fn=trans_func)
    dev_data_loader = create_dataloader(
        dev_ds,
        mode='dev',
        batch_size=args.batch_size,
        batchify_fn=batchify_fn,
        trans_fn=trans_func)
    
    test_data_loader = create_dataloader(
        test_ds,
        mode='test',
        batch_size=args.batch_size,
        batchify_fn=batchify_fn,
        trans_fn=trans_func)

    if args.init_from_ckpt and os.path.isfile(args.init_from_ckpt):
        state_dict = paddle.load(args.init_from_ckpt)
        model.set_dict(state_dict)
    model = paddle.DataParallel(model)

    num_training_steps = len(train_data_loader) * args.epochs

    lr_scheduler = LinearDecayWithWarmup(args.learning_rate, num_training_steps,
                                         args.warmup_proportion)

    # Generate parameter names needed to perform weight decay.
    # All bias and LayerNorm parameters are excluded.
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
        learning_rate=lr_scheduler,
        parameters=model.parameters(),
        weight_decay=args.weight_decay,
        apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.loss.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()

    global_step = 0
    tic_train = time.time()
    for epoch in range(1, args.epochs + 1):
        for step, batch in enumerate(train_data_loader, start=1):
            input_ids, pinyin_ids, labels = batch
            logits = model(input_ids, pinyin_ids)
            loss = criterion(logits, labels)
            probs = F.softmax(logits, axis=1)
            correct = metric.compute(probs, labels)
            metric.update(correct)
            acc = metric.accumulate()

            global_step += 1
            if global_step % 10 == 0 and rank == 0:
                print(
                    "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                    % (global_step, epoch, step, loss, acc,
                       10 / (time.time() - tic_train)))
                tic_train = time.time()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.clear_grad()
            if global_step % 100 == 0 and rank == 0:
                save_dir = os.path.join(args.save_dir, "model_%d" % global_step)
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                print("dev eval:")
                evaluate(model, criterion, metric, dev_data_loader)
                print("test eval:")
                evaluate(model, criterion, metric, test_data_loader)
                # model._layers.save_pretrained(save_dir)
                # tokenizer.save_pretrained(save_dir)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--save_dir", default='./checkpoint', type=str, help="The output directory where the model checkpoints will be written.")
parser.add_argument("--max_seq_length", default=512, type=int, help="The maximum total input sequence length after tokenization. "
    "Sequences longer than this will be truncated, sequences shorter will be padded.")
parser.add_argument("--batch_size", default=2, type=int, help="Batch size per GPU/CPU for training.")
parser.add_argument("--learning_rate", default=2e-5, type=float, help="The initial learning rate for Adam.")
parser.add_argument("--weight_decay", default=0.0001, type=float, help="Weight decay if we apply some.")
parser.add_argument("--epochs", default=10, type=int, help="Total number of training epochs to perform.")
parser.add_argument("--warmup_proportion", default=0.1, type=float, help="Linear warmup proption over the training process.")
parser.add_argument("--init_from_ckpt", type=str, default=None, help="The path of checkpoint to be loaded.")
parser.add_argument("--seed", type=int, default=1000, help="random seed for initialization")
parser.add_argument('--device', choices=['cpu', 'gpu', 'xpu'], default="gpu", help="Select which device to train model, defaults to gpu.")
args = parser.parse_args(args=[])

# ChineseBertModel
CHINESEBERT_PADDLE_PATH = "./pretrain_models/paddle/ChineseBERT-large"
model = ppnlp.transformers.GlyceBertForSequenceClassification.from_pretrained(CHINESEBERT_PADDLE_PATH, num_classes=4)

# ChineseBertTokenizer
tokenizer = ppnlp.transformers.ChineseBertTokenizer(CHINESEBERT_PADDLE_PATH)
do_train(model, tokenizer)

W0916 18:16:56.745079 22892 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 6.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0916 18:16:56.749233 22892 device_context.cc:422] device: 0, cuDNN Version: 8.0.


---> Tensor(shape=[2, 4], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[ 0.29300141,  0.06154867, -0.04159497, -0.24178788],
        [ 0.12031405,  0.07091312,  0.00587054, -0.19262496]])
---> Tensor(shape=[2, 4], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[ 0.13836509, -0.04072359,  0.08384413, -0.26441869],
        [ 0.21721396, -0.22747000,  0.03117523, -0.11488929]])
---> Tensor(shape=[2, 4], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[ 0.17472669, -0.12867226,  0.17266887, -0.13069257],
        [ 0.49496806, -0.18507436, -0.00395256, -0.10577026]])
---> Tensor(shape=[2, 4], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[ 0.29058242, -0.31666705, -0.13665821, -0.17656021],
        [ 0.22873819, -0.01709885, -0.05359051, -0.17607048]])
---> Tensor(shape=[2, 4], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[ 0.09601748,  0.24593727, -0.44436669, -0.41083151],
        [ 0.30488005, -0.021